# Laboratorio 2 - Parte 2

### Modelos no paramétricos

### 2018-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

In [0]:
#Tomado de https://github.com/rramosp/mooc-grader
from Autentication import *#python 3
import inspect, urllib
html, auth_code, userinfo = google_authenticate(PORT_NUMBER=8080)
html

ModuleNotFoundError: No module named 'httplib2'

#### Segundo integrante:

In [0]:
%matplotlib inline
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from __future__ import division

#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")

## Ejercicio 1

Para el problema de clasificación usaremos la base de datos iris, que cargaremos desde la librería sklearn. Responda las siguientes preguntas:

In [2]:
#cargamos la bd iris desde el dataset de sklearn
from sklearn import datasets

iris = datasets.load_iris()

#Esta es la base de datos Iris del UCI Machine Learning Repository 
#https://archive.ics.uci.edu/ml/datasets/iris

X, Y = iris.data, iris.target

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(

In [0]:
xuni = np.unique(X)
clases= np.unique(Y)
print('Hay ' + str(clases) + ' clases' )
tamx = np.size(X[:,1])
print('Hay ' + str(tamx) + ' muestras')

tamy = np.size(X[1,:])
print('Hay ' + str(tamy) + ' caracteristicas')
tmxc = np.unique(Y, return_counts=True)
print('Hay ' + str(tmxc) +"Muestras x clase ", )

Hay [0 1 2] clases
Hay 150 muestras
Hay 4 caracteristicas
Hay (array([0, 1, 2]), array([50, 50, 50], dtype=int64))Muestras x clase 


1.1 ¿Cu&aacute;ntas clases tiene el problema?:
R: Se presentan 3 clases

1.2 ¿Cu&aacute;ntas caracter&iacute;sticas tiene el problema?:
R: Se presentan 4 características

1.3 ¿Cu&aacute;ntas muestras tiene el problema?:
R: Se presentan 150 muestras

1.4 ¿Cu&aacute;ntas muestras por cada clase hay en la base de datos?:
R: Se presentan 50 muestras por cada clase

<b>Nota</b>: Agregue una celda con el código usado para responder cada una de las preguntas anteriores.

## Ejercicio 2

Analice los siguientes métodos a la luz de la teoría vista para los modelos de K vecinos más cercanos (KNN) y complete el código del método KNN.

<b>Nota</b>: Para el cáculo de la distancia entre vectores tienen dos opciones, usar la función scipy.spatial.distance.euclidean o usar la función numpy.linalg.norm. <b>Revisen la documentación </b> para comprender su uso. También serán de utilidad las funciones numpy.sort y numpy.argsort.

In [0]:
from scipy.stats import mode
import operator

#Para calcular el error en los problemas de clasificación
def ErrorClas(Y_lest, Y):
    error = 1 - np.sum(Y_lest == Y)/len(Y)
    
    return error

def distance(X_val, X_train):
    
    MdotXY = np.dot(X_val, X_train.T)
    sumxv = np.sum(X_val * X_val, axis=1).reshape([X_val.shape[0],1])
    sumxt = np.sum(X_train * X_train, axis=1).reshape([1,X_train.shape[0]])
    
    D = np.dot(sumxv, np.ones((1,X_train.shape[0]))) + np.dot(np.ones((X_val.shape[0],1)), sumxt) - 2* MdotXY
    D = np.sqrt(D)
    
    return D

def neighborhood (k, D_argsort, Y_train):
    N = len(D_argsort)    
    kn = D_argsort[ : , 0: k]
    kn = kn.reshape(len(D_argsort),k)
    Y_val = np.zeros((N,k))

  #  Y_val =np.array( [[Y_train[kn[y,x]] for x in range(k)] for y in range(N)])
    for x in range (N):
        for y in range (k):
               Y_val[x][y] = Y_train[kn[x,y]]
    
    return Y_val

def KNN(X_train, Y_train, X_val, k, tipo):
    
    #X_train es la matriz con las muestras de entrenamiento
    #Y_train es un vector con los valores de salida pra cada una de las muestras de entrenamiento
    #X_val es la matriz con las muestras de validación
    #tipo es una bandera que indica si el problema es de regresión o de clasificación.
    tipo = tipo
    k = k    #Parámetro k que equivale al número de vecinos a tener en cuenta para resolver el problema de
               #predicción de la variable de salida
    
    N = np.shape(Y_train)[0]
    Y_val = np.zeros(N).reshape(N,1)
    Y_train = Y_train.reshape(N,1)
    
    D = distance(X_val, X_train)
    D_argsort = np.argsort(D, axis = 1)
    
    LXv = len(X_val)
    
    Y_argsort = neighborhood(k, D_argsort, Y_train)

    Y_val = np.zeros(LXv).T
    

    if(tipo == 'regression'):
        #regresion
        for x in range (Nt):
            average = np.average(Y_argsort[x])
            Y_val[x] = average
            
    elif(tipo == 'classification'):
        #clasificacion
        
        for x in range (LXv):
            mode = stats.mode(Y_argsort[x])           
            Y_val[x] = mode[0].astype(float)
            
    Y_val.reshape(LXv,1)
    
    return Y_val#Debe retornar un vector que contenga las predicciones para cada una de las muestras en X_val, en el mismo orden.  


## Ejercicio 3

Una vez haya completado el codigo del método de KNN, ejecute varias veces el proceso de entrenamiento y evaluacion cambiando el parametro $k$, el cual es el numero de vecinos, y complete la siguiente tabla con los valores del error de clasificación obtenidos:

In [7]:
from numpy import random
from sklearn.model_selection import train_test_split
import math
from scipy import stats

vecinos = 7
tipo = 'classification'

N = np.size(X,0)

#Validamos el modelo
random.seed(1)
Error = np.zeros(4)
for j in range(4):
    
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.25)

    #Normalizamos los datos
    media = np.mean(Xtrain)
    desvia = np.std(Xtrain)
    Xtrain = sc.stats.stats.zscore(Xtrain)
    Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

    Yest = KNN(Xtrain, Ytrain, Xtest, vecinos, tipo)#Complete con el llamado apropiado de la función que Uds diseñaron
    
    #Evaluamos las predicciones del modelo con los datos de test
    Error[j] = ErrorClas(Yest,Ytest)
print('\nError durante la prueba = ' + str(np.mean(Error)) + '+-' + str(np.std(Error)))


Error durante la prueba = 0.2763157894736842+-0.0842516347030638


In [0]:
xTuni = np.unique(Xtrain)
xTestuni= np.unique(Xtest)
print(clases)
tamx = np.size(Xtrain)
#print('Hay ' + str(tamx) + ' muestras entrenamiento')

tamy = np.size(Xtest)
#print('Hay ' + str(tamy) + ' muestras de validacion')

print("Muestras entrenamiento ",Xtrain.shape)
print("Muestras para validación ",Xtest.shape)


[0 1 2]
Muestras entrenamiento  (112, 4)
Muestras para validación  (38, 4)


Responda:

3.1 ¿Qué metodología de validación se usa en el experimento?
R:La metodología de validación implementada es validación cruzada, es este caso  k=4  folds.

3.2 ¿Cuántas muestras se usan para el entrenamiento?
R: 112 muestras para el entrenamiento

3.3 ¿Cuántas muestras se usan para la validación?
R: 38 muestras para el entrenamiento

#### Tabla de resultados

In [0]:
import numpy as np
%matplotlib inline
import pandas as pd
import qgrid
randn = np.random.randn
df_types = pd.DataFrame({
    'Numero de vecinos' : pd.Series(['1', '2', '3', '4', '5', '6', '7'])})
df_types["Error_Prueba"] = ""
df_types["Desviación estándar del error"] = ""
df_types.set_index(['Numero de vecinos'], inplace=True)
df_types["Error_Prueba"][1] = "0.2697"
df_types["Desviación estándar del error"][1] = "0.088"
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

ModuleNotFoundError: No module named 'qgrid'

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [0]:
qgrid_widget.get_changed_df()

,Error_Prueba,Desviación estándar del error
Numero de vecinos,,
1,,
2,0.2697,0.088
3,,
4,,
5,,
6,,
7,,


In [0]:
#1	0.2039	+-0.1137
2	0.2697	+-0.088
3	0.2697	+-0.086
4	0.2763	+-0.0842
5	0.2631	+-0.093
6	0.2763	+-0.0842
7	0.2697	+-0.0880


## Ejercicio 4

A continuación debe completar el código del método parzen para resolver problemas de clasificación con el modelo ventana de Parzen.

In [0]:
#Calcula el kernel gaussiano de x
def kernel_gaussiano(x):
    return np.exp((-0.5)*x**2)

def ParzenWindow(x,Data,h):
    h = h
    Ns = Data.shape[0]
    suma = 0
    for k in range(Ns):
        u = sc.spatial.distance.euclidean(x,Data[k,:])
        suma += kernel_gaussiano(u/h)
    return suma

def ParzenClass(X_train, Y_train, X_val, ancho_h):
    
    Lx = X_val.shape[0]
    Yest = np.zeros(Lx)
    h = ancho_h
    
    # 3 CLASES
    class0 = X_train[Y_train == 0]
    class1 = X_train[Y_train == 1]
    class2 = X_train[Y_train == 2]
    
    z0= np.zeros(Lx)
    z1= np.zeros(Lx)   
    z2= np.zeros(Lx)
    
    for i in range(Lx):        
        z0[i] = ParzenWindow(X_val[i],class0, h)
        z1[i] = ParzenWindow(X_val[i],class1, h)
        z2[i] = ParzenWindow(X_val[i],class2, h)        
        z = np.array([z0[i], z1[i], z2[i]])
        classM = np.argmax(z)
        Yest[i] = classM    
    return Yest#Debe retornar un vector que contenga las predicciones para cada una de las muestras en X_val, en el mismo orden.  

## Ejercicio 5

Ahora debe resolver el mismo problema de clasificación con el método ventana de Parzen. Complete el código ejecute las pruebas para diferentes valores de $h$ y llene la siguiente tabla:

In [0]:
from numpy import random
from sklearn.model_selection import StratifiedKFold
import math

H = 10
#Validamos el modelo
random.seed(1)
Error = np.zeros(4)
skf = StratifiedKFold(n_splits=4)
j = 0
for train, test in skf.split(X, Y):
    Xtrain = X[train,:]
    Ytrain = Y[train]
    Xtest = X[test,:]
    Ytest = Y[test]
    #Normalizamos los datos
    media = np.mean(Xtrain)
    desvia = np.std(Xtrain)
    Xtrain = sc.stats.stats.zscore(Xtrain)
    Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

    Yest = ParzenClass(Xtrain, Ytrain, Xtest,H)#Complete con el llamado apropiado de la función que Uds diseñaron
  
    #Evaluamos las predicciones del modelo con los datos de test
    Error[j] = ErrorClas(Yest,Ytest)
    j += 1
print('\nError durante la prueba = ' + str(np.mean(Error)) + '+-' + str(np.std(Error)))


Error durante la prueba = 0.2216880341880342+-0.04696907336459245


In [0]:
xTuni = np.unique(Xtrain)
xTestuni= np.unique(Xtest)
print(clases)
tamx = np.size(Xtrain)
#print('Hay ' + str(tamx) + ' muestras entrenamiento')

tamy = np.size(Xtest)
#print('Hay ' + str(tamy) + ' muestras de validacion')

print("Muestras entrenamiento ",Xtrain.shape)
print("Muestras para validación ",Xtest.shape)


[0 1 2]
Muestras entrenamiento  (114, 4)
Muestras para validación  (36, 4)


Responda:
    
5.1 ¿Qué metodología de validación se usó en la simulación?:
R: Validación cruzada estratificada con  k=4  particiones, con esto se garantiza que las particiones aleatorias contenga proporción de valores de cada una de las 4 categoría

5.2 ¿Cuántas muestras se usan para el entrenamiento?:
R: 114 muestras para el entrenamiento

5.3 ¿Cuántas muestras se usan para la validación?:
R: 36 muestras para la validación

#### Tabla de resultados

In [0]:
import pandas as pd
import qgrid
randn = np.random.randn
df_types2 = pd.DataFrame({
    'Tamano de ventana' : pd.Series(['0.05', '0.1', '0.5', '1', '2', '5', '10'])})
df_types2["Error_Prueba"] = ""
df_types2["Desviación estándar del error"] = ""
df_types2.set_index(['Tamano de ventana'], inplace=True)
df_types2["Error_Prueba"][1] = "0.222"
df_types2["Desviación estándar del error"][1] = "0.043"
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types2, show_toolbar=False)
qgrid_widget

UWdyaWRXaWRnZXQoZ3JpZF9vcHRpb25zPXsnaGlnaGxpZ2h0U2VsZWN0ZWRSb3cnOiBUcnVlLCAnZnVsbFdpZHRoUm93cyc6IFRydWUsICdyb3dIZWlnaHQnOiAyOCwgJ2VuYWJsZUNvbHVtblLigKY=


Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [0]:
qgrid_widget.get_changed_df()

,Error_Prueba,Desviación estándar del error
Tamaño de ventana,,
0.05,,
0.1,0.222,0.043
0.5,,
1,,
2,,
5,,
10,,


In [0]:
#0.05	0.221	+-0.043
0.1	0.221	+-0.043
0.5	0.281	+-0.040
1	0.280	+-0.031
2	0.241	+-0.058
5	0.221	+-0.046
10	0.221	+-0.046